In [1]:
using MacroEnergy
using MacroEnergy: get_edges, can_retrofit, is_retrofit, new_capacity, retired_capacity
using Gurobi
using JuMP

In [2]:
system = MacroEnergy.load_system(@__DIR__)

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.29 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ W

MacroEnergy.System("/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test", (ConstraintScaling = true, WriteSubcommodities = true, OverwriteResults = false, OutputDir = "results", OutputLayout = "long", AutoCreateNodes = true, AutoCreateLocations = true), Dict{Symbol, DataType}(:CementFuel => CementFuel, :Uranium => Uranium, :CO2Captured => CO2Captured, :Electricity => Electricity, :Hydrogen => Hydrogen, :CO2 => CO2, :Coal => Coal, :LiquidFuels => LiquidFuels, :Cement => Cement, :NaturalGas => NaturalGas…), Dict{Symbol, MacroEnergy.TimeData}(:Cement => MacroEnergy.TimeData{Cement}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :NaturalGas => MacroEnergy.TimeData{NaturalGas}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :CementFuel => MacroEnergy.TimeData{CementFuel}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :CO2 => MacroEnergy.TimeData{CO2}

In [2]:
model = MacroEnergy.generate_model(system)
MacroEnergy.set_optimizer(model, Gurobi.Optimizer)
MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
MacroEnergy.optimize!(model)

┌ Info: Generating model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:3
┌ Info:  -- Adding linking variables
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:15
┌ Info:  -- Defining available capacity
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:18
┌ Info:  -- Generating planning model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:21
┌ Info:  -- Generating operational model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:24
┌ Info:  -- Model generation complete. It took 12.01 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:29


Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 840988 rows, 551927 columns and 2845961 nonzeros
Model fingerprint: 0x4a111cd2
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 4e+06]
Presolve removed 394763 rows and 189997 columns
Presolve time: 0.57s
Presolved: 446225 rows, 361930 columns, 2196057 nonzeros
Ordering ti

In [11]:
MacroEnergy.value(system.assets[2].cement_edge.retrofitted_capacity)

0.03450592361380814

In [12]:
MacroEnergy.value(system.assets[5].cement_edge.new_capacity)

0.03450592361380814

In [ ]:
MacroEnergy.compute_conflict!(model)
list_of_conflicting_constraints = MacroEnergy.ConstraintRef[];
for (F, S) in Macro.list_of_constraint_types(model)
    for con in Macro.JuMP.all_constraints(model, F, S)
        if Macro.JuMP.get_attribute(con, Macro.MOI.ConstraintConflictStatus()) == Macro.MOI.IN_CONFLICT
            push!(list_of_conflicting_constraints, con)
        end
    end
end
display(list_of_conflicting_constraints)

In [ ]:
include("../helper_functions.jl")
using DataFrames

In [ ]:
nodes = Node[x for x in system.locations if x isa MacroEnergy.Node]

In [ ]:
co2_nodes = MacroEnergy.get_nodes_sametype(nodes, CO2)

In [ ]:
case = "0pct_co2_reduction"
results_dir = joinpath(@__DIR__,  "results")
mkpath(results_dir)
save_results(results_dir, case, system)

In [ ]:
model[:cRetrofitCapacity]

In [ ]:
function get_unique_retrofit_tuples(system::MacroEnergy.System)
    # Returns a list of retrofit clusters in the system, described as tuples: (retrofit_id, location)
    can_retrofit_ids = []
    is_retrofit_ids = []
    for e in get_edges(system)
        if can_retrofit(e)
            push!(can_retrofit_ids, (e.retrofit_id, e.location))
        end
        if is_retrofit(e)
            push!(is_retrofit_ids, (e.retrofit_id, e.location))
        end
    end
    @assert unique(can_retrofit_ids) == unique(is_retrofit_ids)
    retrofit_ids = unique(can_retrofit_ids)
    return retrofit_ids
end

function get_can_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can be retrofitted with a given retrofit_id and location
    can_retrofit_edges = []
    for edge in edges
        if (can_retrofit(edge) == true) && (edge.retrofit_id == retrofit_id) && (edge.location == location)
            push!(can_retrofit_edges, edge)
        end
    end
    return can_retrofit_edges
end

function get_is_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can retrofit other edges with a given retrofit_id and location
    is_retrofit_edges = []
    for edge in edges
        if (is_retrofit(edge) == true) && (edge.retrofit_id == retrofit_id) && (edge.location == location)
            push!(is_retrofit_edges, edge)
        end
    end
    return is_retrofit_edges
end

In [ ]:
edges = get_edges(system)

function get_can_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can be retrofitted with a given retrofit_id and location
    can_retrofit_edges = []
    for e in edges
        if (can_retrofit(e) == true) && (retrofit_id(e) == retrofit_id) && (location(e) == location)
            push!(can_retrofit_edges, e)
        end
    end
    return can_retrofit_edges
end

function get_is_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can retrofit other edges with a given retrofit_id and location
    is_retrofit_edges = []
    for e in edges
        if (is_retrofit(e) == true) && (retrofit_id(e) == retrofit_id) && (location(e) == location)
            push!(is_retrofit_edges, e)
        end
    end
    return is_retrofit_edges
end

get_is_retrofit_edges (generic function with 1 method)

In [ ]:
retrofit_id = 1
location = "MA"
e = edges[3]
location(e)

MethodError: MethodError: objects of type String are not callable
The object of type `String` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

In [ ]:
edges = get_edges(system)
retrofit_tuples = get_unique_retrofit_tuples(system)

@constraint(model, cRetrofitCapacity[(retrofit_id, retrofit_location) in retrofit_tuples],
    sum(new_capacity(e) for e in get_can_retrofit_edges(edges, retrofit_id, retrofit_location)) <=
    sum(retired_capacity(e) for e in get_is_retrofit_edges(edges, retrofit_id, retrofit_location))
)

In [ ]:
edges = get_edges(system)

In [ ]:
nodes = Node[x for x in system.locations if x isa MacroEnergy.Node]
co2_node = MacroEnergy.get_nodes_sametype(nodes, CO2)[1]

In [ ]:
system.assets[1].cement_edge.end_vertex

In [4]:
df = CSV.read("results/test_new_cementplant_config/test_new_cementplant_config_flows.csv", DataFrame)

UndefVarError: UndefVarError: `CSV` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: CSV is loaded but not imported in the active module Main.